<a href="https://colab.research.google.com/github/pfolaa/dsti-labs/blob/main/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob

In [ ]:
#cd /content/drive/MyDrive/datasets/

/content/drive/MyDrive/datasets


## 1- Data acquisition

###1.1- Get files from repository 

In [131]:
#!wget https://liberta-leasing-fraud.s3.eu-west-1.amazonaws.com/nirra-log-bot.zip

### 1.2 Extract zip file

In [ ]:
from zipfile import ZipFile
#ZipFile("nirra-log-bot.zip").extractall('/content/drive/My Drive/datasets/')

### 1.3 Read all files and insert into a csv file

In [159]:
import os,json
import pandas as pd

# read json file
path_to_json = '/content/drive/My Drive/datasets/nirra-log-bot/'

# read all json files
for file_name in [file for file in os.listdir(path_to_json) if file.endswith('.json')]:
  with open(path_to_json + file_name) as json_file:
    data = json.load(json_file)
    df_raw = pd.DataFrame.from_records(data)

# convert file to csv
df_raw.to_csv('/content/drive/My Drive/datasets/nirra-log-bot/csv/file_name.csv', sep=';')

In [160]:
df_raw.head()

,type,subtype,text,ts,bot_id
0,message,bot_message,"[info] - [""[OKRA WEBHOOK]:"",""{\""method\"":\""PAY...",1630911747.008800,B01RC3AKYG6
1,message,bot_message,"[info] - [""[API REQUEST]: 08134676606, /client...",1630911747.008900,B01RC3AKYG6
2,message,bot_message,"[info] - [""[API REQUEST]: 08134676606, /client...",1630911749.009000,B01RC3AKYG6
3,message,bot_message,"[info] - [""[API REQUEST]: 08134676606, /client...",1630911754.009100,B01RC3AKYG6
4,message,bot_message,"[info] - [""[API REQUEST]: 08134676606, /client...",1630911754.009200,B01RC3AKYG6


### Get total number of @ inside a text column

In [145]:
 df_raw['text'].str.contains("@").sum() # search total number of @ within text column

37

In [161]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4953 entries, 0 to 4952
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   type     4953 non-null   object
 1   subtype  4953 non-null   object
 2   text     4953 non-null   object
 3   ts       4953 non-null   object
 4   bot_id   4953 non-null   object
dtypes: object(5)
memory usage: 193.6+ KB


### Test a slicing

In [25]:
# slicing
#df_raw[df_raw['text'].str.contains("@")] # masque à l'intérieur des crochets

In [147]:
df_raw[df_raw['text'] == None]

,type,subtype,text,ts,bot_id


### Used in case of WALLET SUCCESS

In [163]:
# when type request is WALLET SUCCESS
import regex
import json

json_text = df_raw['text'][841]
pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
resul_patt = pattern.findall(json_text)
res = resul_patt[0].replace("\\", " ")
s = json.loads(res)
print(s.get('account_number '))
print(s.get('account_name '))
print(s.get('bvn '))
print(s.get('requestSuccessful '))
print(s.get('responseMessage '))
print(s.get('responseCode '))



9980432021 
LIBERTA(Kayode  Ajao) 
22213975706 
True
Reserved Account Generated Successfully 
00 


### Used in case of SMS SUCCESS

In [164]:
# use this function when type request is SMS SUCCESS

import regex
import json

json_text = df_raw['text'][12]
pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
resul_patt = pattern.findall(json_text)
res = resul_patt[0].replace("\\", " ")

s = json.loads(res)
cost = s.get('response ').get('cost ')
status = s.get('response ').get('status ')
totalsent = s.get('response ').get('totalsent ')
print(cost)
print(status)
print(totalsent)
res

2
SUCCESS 
1


'{ "response ":{ "status ": "SUCCESS ", "totalsent ":1, "cost ":2}}'

### Used in case of SMS PAYLOAD

In [165]:
# use this function when type request is SMS PAYLOAD

import regex
import json

json_text = df_raw['text'][11]
pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
resul_patt = pattern.findall(json_text)
res = resul_patt[0].replace("\\", " ")
s = json.loads(res)
phone_number = s.get('phone ').replace(" ", "")
message = s.get('message ')
phone_number +' '+ message



'2347035864882 To continue your signup, use this OTP 169 600 '

### Handle row with type request "LEADWAY SUCCESS" and concatenate rows

In [166]:
import re


def parseLeadwaySuccessRow():
  first_index = 0
  last_index = 0
  text_leadway = ''
  for index, row in df_raw.iterrows():
      text_row = row['text']  
      if re.search('LEADWAY SUCCESS', text_row):
        text_leadway = text_row
        first_index = index
        first_index +=1
        new_df = df_raw[first_index:]
        for first_index, new_row in new_df.iterrows():
          xxx = new_row['text']      
          if not xxx.startswith('['):          
            first_index += 1
            text_leadway = text_row + xxx       
          elif xxx.startswith('['):
            last_index = first_index-1
            break


  return text_leadway


### Parse row Leadway Success to json

In [ ]:
# use this function when type request is LEADWAY SUCCESS
import regex
import json

text_leadway = parseLeadwaySuccessRow()
pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
resul_patt = pattern.findall(text_leadway)
resul_patt[0] = resul_patt[0].replace("\\", "")
x = resul_patt[0].replace("make,", "")
y = x.replace('""makeName"', '"makeName"')
z = json.loads(y)
vehicleMake = z.get('vehicleMake')
for index in range(len(vehicleMake)):
   print(vehicleMake[index].get('id'))
   print(vehicleMake[index].get('makeName'))

In [182]:
res_leadway = parseLeadwaySuccessRow()
#res_leadway

### Drop concatenated rows!

In [168]:
df_raw = df_raw.drop(first_index)
df_raw = df_raw.drop(last_index)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4951 entries, 0 to 4952
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   type     4951 non-null   object
 1   subtype  4951 non-null   object
 2   text     4951 non-null   object
 3   ts       4951 non-null   object
 4   bot_id   4951 non-null   object
dtypes: object(5)
memory usage: 232.1+ KB


### Check rows that do not start with [ and drop them

In [169]:
 for index, row in df_raw.iterrows():
    str_text_1 = row['text']
    if not str_text_1.startswith('['):
      df_raw = df_raw.drop(index)

df_raw.info()    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4946 entries, 0 to 4952
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   type     4946 non-null   object
 1   subtype  4946 non-null   object
 2   text     4946 non-null   object
 3   ts       4946 non-null   object
 4   bot_id   4946 non-null   object
dtypes: object(5)
memory usage: 231.8+ KB


### Handle log level "Error"

In [185]:
import regex
import json
import re

json_text = df_raw['text'][2355].replace('"', "'")
pattern = regex.compile(r"{?[a-z :A-Z 0-9\\,=_`']+selfie")
resul_patt = pattern.findall(json_text)
res = resul_patt[0].replace("\\", " ")
res = res.replace("'name'", "name").replace("`", "").replace("'18'", "18").replace("'monthly'", "monthly")
res = res+'"}'
res = res.replace("\'", '"')
res
s = json.loads(res)
print(s.get('sql '))
print(s.get('code '))
print(s.get('errno '))
print(s.get('index '))
print(s.get('sqlMessage '))
print(s.get('sqlState '))


INSERT INTO preapplications Set product = 11, loan_amount = 5000, tenor = 1, loan_purpose = 18, tenor_type = monthly, selfie
ER_BAD_NULL_ERROR 
1048
0
Column name cannot be null 
23000 


In [172]:
import regex
import json

def parseErrorRow(error_row):
  error_row.replace('"', "'")
  pattern = regex.compile(r"{?[a-z :A-Z 0-9\\,=_`']+selfie")
  resul_patt = pattern.findall(error_row)
  res = resul_patt[0].replace("\\", " ")
  res = res.replace("'name'", "name").replace("`", "").replace("'18'", "18").replace("'monthly'", "monthly")
  res = res+'"}'
  res = res.replace("'", '"')
  s = json.loads(res)
  return s

### Function to convert data to timestamp

In [173]:
import datetime

# function to convert date to Timestamp
def convertToTimestamp(str):
  element = datetime.datetime.strptime(str,"%Y-%m-%dT%H:%M:%S.%fZ")
  return datetime.datetime.timestamp(element)

In [174]:
timestamp = convertToTimestamp('2021-09-05T07:03:55.223Z')
timestamp

1630825435.223

## 2 Data model
### 2.1 Parse rows of dataset

In [186]:
import re
import regex
import json

# parse row of text column

def parserow(df_):
  log_level_col = []
  api_request_col = []
  type_request_col = []
  phone_Col = []
  date_col = []
  endpoint_Col = []
  email_col = []
  message_sms_payload_col = []
  totalsent_col = []
  cost_col = []
  status_col = []
  account_number_col = []
  account_name_col = []
  bvn_col = []
  requestSuccessful_col = []
  responseMessage_col = []
  responseCode_col = []
  vehicle_make_id_col = []
  vehicle_make_name_col = []

  for index, row in df_.iterrows():
    str_text = row['text']
        
    # check if the row contains "info" string
    if re.search('info', str_text):
        log_level = re.search('info', str_text)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)        
        # check if the row contains an email address 
        if 'mailto' in str_text:
            if re.search('API REQUEST', str_text):
                type_of_request = re.search('API REQUEST', str_text)
                phone_or_email = re.search('[\w.+-]+@[\w-]+\.[\w.-]+', str_text)    # extract address email for API REQUEST               
                endpoint = re.search('/[/a-z 0-9?=&;/_A-Z+]+', str_text) # extract endpoint
                pattern = "(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d).(\d{3})*[a-zA-Z]"
                datepattern = re.compile("(?:%s)"%(pattern))
                datematcher = datepattern.search(str_text)  # extract date
                message_sms_payload_col.append(None) # in this case there is no message sms payload 
                totalsent_col.append(None)
                cost_col.append(None)
                status_col.append(None)
                account_number_col.append(None)
                bvn_col.append(None)
                requestSuccessful_col.append(None)
                responseMessage_col.append(None)
                responseCode_col.append(None)
                vehicle_make_id_col.append(None)
                vehicle_make_name_col.append(None)
                account_name_col.append(None)
                try:
                  type_request_col.append(type_of_request.group(0)) # add type request inside type request column
                except AttributeError:
                  type_request_col.append(None)               
                try:
                  email_col.append(phone_or_email.group(0)) # add email inside email column
                  phone_Col.append(None)  # in this case there is no phone number
                except AttributeError:
                  email_col.append(None)
                try:
                  endpoint_Col.append(endpoint.group(0)) # add endpoint inside endpoint column
                except AttributeError:
                  endpoint_Col.append(None)
                try:
                  date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                except AttributeError:
                  date_col.append(None)
              
            elif re.search('CLIENT MOBILE LOGIN', str_text):   # CLIENT MOBILE LOGIN with email address
                  type_of_request = re.search('CLIENT MOBILE LOGIN', str_text)
                  phone_or_email = re.search('[\w.+-]+@[\w-]+\.[\w.-]+', str_text)    # extract address email for CLIENT MOBILE LOGIN
                  pattern = "(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d).(\d{3})*[a-zA-Z]"
                  datepattern = re.compile("(?:%s)"%(pattern))
                  datematcher = datepattern.search(str_text)  # extract date for CLIENT MOBILE LOGIN type request
                  message_sms_payload_col.append(None) # in this case there is no message sms payload 
                  totalsent_col.append(None)
                  cost_col.append(None)
                  status_col.append(None)
                  account_number_col.append(None)
                  bvn_col.append(None)
                  requestSuccessful_col.append(None)
                  responseMessage_col.append(None)
                  responseCode_col.append(None)
                  vehicle_make_id_col.append(None)
                  vehicle_make_name_col.append(None)
                  account_name_col.append(None)
                  endpoint_Col.append(None)
                  try:
                    type_request_col.append(type_of_request.group(0)) # add type request inside type request column
                  except AttributeError:
                    type_request_col.append(None) 
                  try:
                    email_col.append(phone_or_email.group(0)) # add email inside email column
                    phone_Col.append(None)  # in this case there is no phone number
                  except AttributeError:
                    email_col.append(None)
                  try:
                    date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                  except AttributeError:
                    date_col.append(None)

        elif 'mailto' not in str_text:
            if re.search('API REQUEST', str_text):
                type_of_request = re.search('API REQUEST', str_text)
                phone_or_email = re.search('[0-9]+', str_text)    # extract a phone number for API REQUEST
                endpoint = re.search('/[/a-z 0-9?=&;/_A-Z+]+', str_text) # extract endpoint
                pattern = "(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d).(\d{3})*[a-zA-Z]"
                datepattern = re.compile("(?:%s)"%(pattern))
                datematcher = datepattern.search(str_text)  # extract date
                message_sms_payload_col.append(None) # in this case there is no message sms payload
                totalsent_col.append(None)
                cost_col.append(None)
                status_col.append(None)
                account_number_col.append(None)
                bvn_col.append(None)
                requestSuccessful_col.append(None)
                responseMessage_col.append(None)
                responseCode_col.append(None)
                vehicle_make_id_col.append(None)
                vehicle_make_name_col.append(None)
                account_name_col.append(None)
                try:
                  phone_Col.append(phone_or_email.group(0)) # add phone number inside phone number column
                  email_col.append(None) # in this case there is no email address
                except AttributeError:
                  phone_Col.append(None)
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:
                  endpoint_Col.append(endpoint.group(0)) # add endpoint inside endpoint column
                except AttributeError:
                  endpoint_Col.append(None)
                try:
                  date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                except AttributeError:
                  date_col.append(None)
            elif re.search('CLIENT MOBILE LOGIN', str_text): # when type request is CLIENT MOBILE LOGIN, there is no EndPoint
                type_of_request = re.search('CLIENT MOBILE LOGIN', str_text)
                phone_or_email = re.search('[0-9]+', str_text)    # extract a phone number for CLIENT MOBILE LOGIN
                pattern = "(\d{4})-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d).(\d{3})*[a-zA-Z]"
                datepattern = re.compile("(?:%s)"%(pattern))
                datematcher = datepattern.search(str_text)  # extract date
                endpoint_Col.append(None) # in this case there is no endpoint
                message_sms_payload_col.append(None) # in this case there is no message sms payload
                email_col.append(None)
                totalsent_col.append(None)
                cost_col.append(None)
                status_col.append(None)
                account_number_col.append(None)
                bvn_col.append(None)
                requestSuccessful_col.append(None)
                responseMessage_col.append(None)
                responseCode_col.append(None)
                vehicle_make_id_col.append(None)
                vehicle_make_name_col.append(None)
                account_name_col.append(None)
                try:
                  phone_Col.append(phone_or_email.group(0))
                except AttributeError:
                  phone_Col.append(None)
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:
                  date_col.append(convertToTimestamp(datematcher.group(0))) # convert date to timestamp and add it inside date column
                except AttributeError:
                  date_col.append(None)

            elif re.search('SMS PAYLOAD', str_text):
                type_of_request = re.search('SMS PAYLOAD', str_text)            
                pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
                resul_patt_sms_payload = pattern.findall(str_text)
                res_sms_payload = resul_patt_sms_payload[0].replace("\\", " ")
                sms_payload = json.loads(res_sms_payload)
                phone_number = sms_payload.get('phone ').replace(" ", "")
                message = sms_payload.get('message ')
                totalsent_col.append(None)
                cost_col.append(None)
                status_col.append(None)
                account_number_col.append(None)
                bvn_col.append(None)
                requestSuccessful_col.append(None)
                responseMessage_col.append(None)
                responseCode_col.append(None)
                vehicle_make_id_col.append(None)
                vehicle_make_name_col.append(None)
                account_name_col.append(None)
                email_col.append(None)
                endpoint_Col.append(None)
                date_col.append(None)
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:
                  phone_Col.append(phone_number)
                except AttributeError:
                  phone_Col.append(None)
                try:
                  message_sms_payload_col.append(message)
                except AttributeError:
                  message_sms_payload_col.append(None)
                
            elif re.search('SMS SUCCESS', str_text): 
                type_of_request = re.search('SMS SUCCESS', str_text)
                pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
                resul_patt_sms_success = pattern.findall(str_text)
                res_sms_success = resul_patt_sms_success[0].replace("\\", " ")
                sms_success = json.loads(res_sms_success)
                cost = sms_success.get('response ').get('cost ')
                status = sms_success.get('response ').get('status ')
                totalsent = sms_success.get('response ').get('totalsent ')
                message_sms_payload_col.append(None) # in this case there is no message sms payload              
                account_number_col.append(None)
                bvn_col.append(None)
                requestSuccessful_col.append(None)
                responseMessage_col.append(None)
                responseCode_col.append(None)
                vehicle_make_id_col.append(None)
                vehicle_make_name_col.append(None)
                account_name_col.append(None)
                email_col.append(None)
                phone_Col.append(None)
                endpoint_Col.append(None)
                date_col.append(None)
                try:
                  type_request_col.append(type_of_request.group(0))
                except AttributeError:
                  type_request_col.append(None)
                try:                 
                  totalsent_col.append(totalsent)
                except AttributeError:
                  totalsent_col.append(None)
                try:                 
                  cost_col.append(cost)
                except AttributeError:
                  cost_col.append(None)
                try:                 
                  status_col.append(status)
                except AttributeError:
                  status_col.append(None)
            elif re.search('WALLET PAYLOAD', str_text):
                  type_of_request = re.search('WALLET PAYLOAD', str_text)
                  message_sms_payload_col.append(None) # in this case there is no message sms payload
                  totalsent_col.append(None)
                  cost_col.append(None)
                  status_col.append(None)
                  account_number_col.append(None)
                  bvn_col.append(None)
                  requestSuccessful_col.append(None)
                  responseMessage_col.append(None)
                  responseCode_col.append(None)
                  vehicle_make_id_col.append(None)
                  vehicle_make_name_col.append(None)
                  account_name_col.append(None)
                  email_col.append(None)
                  phone_Col.append(None)
                  endpoint_Col.append(None)
                  date_col.append(None)
                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)
            elif re.search('WALLET SUCCESS', str_text):
                  type_of_request = re.search('WALLET SUCCESS', str_text)
                  pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
                  resul_patt_wallet_success = pattern.findall(str_text)
                  res_wallet_success = resul_patt_wallet_success[0].replace("\\", " ")
                  wallet_success = json.loads(res_wallet_success)
                  account_number_col.append(wallet_success.get('account_number '))
                  account_name_col.append(wallet_success.get('account_name '))
                  bvn_col.append(wallet_success.get('bvn '))
                  requestSuccessful_col.append(wallet_success.get('requestSuccessful '))
                  responseMessage_col.append((wallet_success.get('responseMessage ')))
                  responseCode_col.append((wallet_success.get('responseCode ')))
                  message_sms_payload_col.append(None) # in this case there is no message sms payload
                  totalsent_col.append(None)
                  cost_col.append(None)
                  status_col.append(None)
                  vehicle_make_id_col.append(None)
                  vehicle_make_name_col.append(None)
                  email_col.append(None)
                  phone_Col.append(None)
                  endpoint_Col.append(None)
                  date_col.append(None)
                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)
            elif re.search('LEADWAY SUCCESS', str_text):
                  text_leadway = parseLeadwaySuccessRow()
                  pattern = regex.compile(r'\{(?:[^{}]|(?R))*}')
                  resul_patt_leadway_success = pattern.findall(text_leadway)
                  resul_patt_leadway_success[0] = resul_patt_leadway_success[0].replace("\\", "")
                  replace_make = resul_patt_leadway_success[0].replace("make,", "")
                  replace_makeName = replace_make.replace('""makeName"', '"makeName"')
                  leadway_json = json.loads(replace_makeName)
                  vehicleMake = leadway_json.get('vehicleMake')
                  try:
                    type_request_col.append(type_of_request.group(0))
                  except AttributeError:
                    type_request_col.append(None)
                  for index in range(len(vehicleMake)):
                      vehicle_make_id_col.append(vehicleMake[index].get('id'))
                      vehicle_make_name_col.append(vehicleMake[index].get('makeName'))

                  type_of_request = re.search('LEADWAY SUCCESS', str_text)
                  message_sms_payload_col.append(None) # in this case there is no message sms payload
                  totalsent_col.append(None)
                  cost_col.append(None)
                  status_col.append(None)
                  account_number_col.append(None)
                  bvn_col.append(None)
                  requestSuccessful_col.append(None)
                  responseMessage_col.append(None)
                  responseCode_col.append(None)
                  account_name_col.append(None)
                  email_col.append(None)
                  phone_Col.append(None)
                  endpoint_Col.append(None)
                  date_col.append(None)                    
    elif re.search('error', str_text):
        log_level = re.search('error', str_text)

        message_sms_payload_col.append(None) # in this case there is no message sms payload
        totalsent_col.append(None)
        cost_col.append(None)
        status_col.append(None)
        vehicle_make_id_col.append(None)
        vehicle_make_name_col.append(None)
        email_col.append(None)
        phone_Col.append(None)
        endpoint_Col.append(None)
        date_col.append(None)
        bvn_col.append(None)
        requestSuccessful_col.append(None)
        responseMessage_col.append(None)
        responseCode_col.append(None)
        account_name_col.append(None)
        account_number_col.append(None)
        try:
          log_level_col.append(log_level.group(0))
        except AttributeError:
          log_level_col.append(None)

        if re.search('LOAN ERROR', str_text):
            type_of_request = re.search('LOAN ERROR', str_text)
            pattern = regex.compile(r"{?[a-z :A-Z 0-9\\,=_`']+selfie")
            resul_patt_loan_error = pattern.findall(str_text)
            loan_error = resul_patt_loan_error[0].replace("\\", " ")
            loan_error = loan_error.replace("'name'", "name").replace("`", "").replace("'18'", "18").replace("'monthly'", "monthly")
            loan_error = loan_error+'"}'
            loan_error = loan_error.replace("\'", '"')
            s_loan_error = json.loads(loan_error)
            s_loan_error.get('sql ')
            s_loan_error.get('code ')
            s_loan_error.get('errno ')
            s_loan_error.get('index ')
            s_loan_error.get('sql ')
            s_loan_error.get('sqlMessage ')
            s_loan_error.get('sqlState ')

            try:
                type_request_col.append(type_of_request.group(0))
            except AttributeError:
                type_request_col.append(None)

    
        elif re.search('OKRA PAYLOAD', str_text): # Nothing
          type_of_request = re.search('OKRA PAYLOAD', str_text)
        elif re.search('OKRA WEBHOOK', str_text):
          type_of_request = re.search('OKRA WEBHOOK', str_text)
        elif re.search('OKRA SUCCESS', str_text):   # Nothing
          type_of_request = re.search('OKRA SUCCESS', str_text)
        elif re.search('VTPASS SUCCESS', str_text):   # Nothing
          type_of_request = re.search('VTPASS SUCCESS', str_text)  


  type_request_col.append(None)
  type_request_col.append(None)

  phone_Col.append(None)
  phone_Col.append(None)

  date_col.append(None)
  date_col.append(None)

  endpoint_Col.append(None)
  endpoint_Col.append(None)




  email_col.append(None)
  email_col.append(None)

  message_sms_payload_col.append(None)
  message_sms_payload_col.append(None)

  totalsent_col.append(None)
  totalsent_col.append(None)

  cost_col.append(None)
  cost_col.append(None)

  status_col.append(None)
  status_col.append(None)

  account_number_col.append(None)
  account_number_col.append(None)

  account_name_col.append(None)
  account_name_col.append(None)

  bvn_col.append(None)
  bvn_col.append(None)

  requestSuccessful_col.append(None)
  requestSuccessful_col.append(None)

  responseMessage_col.append(None)
  responseMessage_col.append(None)

  responseCode_col.append(None)
  responseCode_col.append(None)

  vehicle_make_id_col.append(None)
  vehicle_make_id_col.append(None)
  vehicle_make_id_col.append(None)

  vehicle_make_name_col.append(None)
  vehicle_make_name_col.append(None)
  vehicle_make_name_col.append(None)


  df_['Type_Request'] = type_request_col
  df_['Phone_Number'] = phone_Col
  df_['Date'] = date_col
  df_['EndPoint'] = endpoint_Col
  df_['Log_Level'] = log_level_col
  df_['Email'] = email_col
  df_['Message SMS Payload'] = message_sms_payload_col
  df_['Total Sent'] = totalsent_col
  df_['Cost'] = cost_col
  df_['Status'] = status_col
  df_['Account Number'] = account_number_col
  df_['Account Name'] = account_name_col
  df_['BVN'] = bvn_col
  df_['Request Successful'] = requestSuccessful_col
  df_['Response Message'] = responseMessage_col
  df_['Response Code'] = responseCode_col
  df_['Vehicle Make ID'] = vehicle_make_id_col
  df_['Vehicle Make Name'] = vehicle_make_name_col

  return df_




In [ ]:
df_parserow = parserow(df_raw)
df_parserow.head()